In [94]:
import sys
sys.path.append('..')

import gc
import webdataset as wds
import braceexpand
import torch
import json
import os

from model.baseline_3d_cnn import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
data_dir = '../data'
shards_dir = os.path.join(data_dir, 'shards')

# Opening JSON file
with open('../parameters.json') as json_file:
    parameters = json.load(json_file)

parameters

{'batch_size': 32}

In [87]:
def test_model():
    x = torch.zeros((32, 1, 12, 512, 512))  
    model = baseline_3DCNN(in_num_ch=1)
    scores = model(x)
    print(scores.size()) 
# gc.collect()
# test_model()

In [88]:
urls = [os.path.join(shards_dir, it) for it in os.listdir(shards_dir)]

dataset = (
    wds
    .WebDataset(urls)
    .shuffle(batch_size)
    .decode('torch')
    .to_tuple('data.pyd', 'target.pyd')
)

for image, target in islice(dataset, 0, 2):
    print(image.shape)

(12, 512, 512)
(12, 512, 512)
(12, 512, 512)


In [109]:
USE_GPU = True
dtype = torch.float64 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [91]:
loader_train = torch.utils.data.DataLoader(dataset.batched(parameters['batch_size']), num_workers=0, batch_size=None)


0
torch.Size([32, 12, 512, 512]) torch.Size([32, 6])

1
torch.Size([32, 12, 512, 512]) torch.Size([32, 6])



In [ ]:
def train(model, optimizer, loader_train, epochs=1, device=device, print_every=1):
    """
    Train a model.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()